In [1]:
from ioMicroSimple import *

In [5]:
from tqdm import tqdm
all_fls_col1 = glob.glob(r'Z:\MERFISHp\12_16_2025_BigSlideTest\mermake_fits\*--H1_MER_set*--col2__Xhfits.npz')
base_folder = r'Z:\MERFISHp\12_16_2025_BigSlideTest\H1_MER'
fovs=  [os.path.basename(fl).split('--')[0] for fl in all_fls_col1]
pos = [get_pos(base_folder+get_set(fov)+os.sep+fov) for fov in tqdm(fovs)]
pix_size=[0.4,0.1083333,0.1083333]
from scipy.spatial import KDTree
pos_pix = np.array(pos)/pix_size[-2:]
#pos_pix=pos_pix[:,::-1]*[1,-1]
tree = KDTree(pos_pix)
dist_pix = 2800*1.25#3500
neigs = tree.query_ball_point(pos_pix,dist_pix)
#.query()

100%|██████████████████████████████████████████████████████████████████████████████| 6302/6302 [03:29<00:00, 30.12it/s]


In [6]:
MA = np.load(r'Z:\MERFISHp\12_16_2025_BigSlideTest\MasterColor1TransformNew.npy')

In [10]:
dic_pair = {}

In [15]:
for ifovf in tqdm(np.arange(len(neigs))):
    Xs = []
    neigs_ = neigs[ifovf]
    for ifov in neigs_:
        Xh = np.load(all_fls_col1[ifov])['Xh']
        X = Xh[Xh[:,-4]>0.5][:,:3]
        if len(X)>0:
            X[:,:3] = apply_colorcor(X[:,:3],MA)
            #X = X[np.all(X<[100,2800,2800],axis=-1)]
            X[:,1:3] = X[:,1:3]+pos_pix[ifov]-pos_pix[neigs_[0]]
            Xs.append(X)
    X0 = Xs[list(neigs_).index(ifovf)]
    tzxys_cts = []
    tzxys = []
    for ix1,X1 in zip(neigs_,Xs[:]):
        if (ifovf,ix1) not in dic_pair:
            tzxy = np.array([np.nan,np.nan,np.nan])
            ncts=0
            if len(X1)>0 and len(X0)>0:
                try:
                    tzxy,ncts = get_best_translation_points(X0,X1,resc=5,target=3,constr=50,return_counts=True)
                except:
                    pass
            tzxy = np.array(tzxy)
            tzxys.append(tzxy)
            dic_pair[(ifovf,ix1)]=[tzxy,ncts]
            dic_pair[(ix1,ifovf)]=[-tzxy,ncts]

100%|████████████████████████████████████████████████████████████████████████████| 6302/6302 [3:20:34<00:00,  1.91s/it]


In [16]:
import pickle
fl_save = os.path.dirname(os.path.dirname(all_fls_col1[0]))+os.sep+'dic_pairs_drift.pkl'
pickle.dump([fovs,pos_pix,dic_pair],open(fl_save,'wb'))

In [17]:
dic_drft = {key:dic_pair[key][0] for key in dic_pair if dic_pair[key][1]>100}
len(dic_drft),len(dic_pair)

(28714, 30316)

In [18]:
def change_pos(ifv,txyz,pos_pix_f,dic_drftf):
    pos_pix_f[ifv]-=txyz[1:]
    keys_neighs = np.array(list(dic_drftf.keys()))
    keysifv = keys_neighs[keys_neighs[:,0]==ifv]
    for key in keysifv:
        dic_drftf[tuple(key)]-=txyz
    keysifv = keys_neighs[keys_neighs[:,1]==ifv]
    for key in keysifv:
        dic_drftf[tuple(key)]+=txyz
    return pos_pix_f,dic_drftf
def get_tzxy_added(ifv,added,dic_drftf):
    keys_neighs = np.array(list(dic_drftf.keys()))
    keysifv = keys_neighs[keys_neighs[:,0]==ifv]
    drifts = np.array([dic_drftf[tuple(key)] for key in keysifv])
    
    keep = np.in1d(keysifv[:,-1],added)
    if np.sum(keep)==0:
        tzxy = np.array([0]*3)
    else:
        tzxy = np.nanmean(drifts[keep],axis=0)
    if np.isnan(tzxy[0]):
        tzxy = np.array([0]*3)
    return tzxy


### Run mean field #####################################
from tqdm import tqdm
pos_fovs = pos_pix
pos_fovs_f = np.array(pos_pix,dtype=float)
dic_drftf = {key:np.array(dic_drft[key],dtype=float) for key in dic_drft}
for i in range(10):
    added = []
    for ifv in tqdm(np.arange(len(pos_fovs_f))):
        txyz = get_tzxy_added(ifv,added,dic_drftf)
        change_pos(ifv,txyz,pos_fovs_f,dic_drftf)
        added.append(ifv)
    added = []
    for ifv in tqdm(np.arange(len(pos_fovs_f))[::-1]):
        txyz = get_tzxy_added(ifv,added,dic_drftf)
        change_pos(ifv,txyz,pos_fovs_f,dic_drftf)
        added.append(ifv)

  0%|                                                                                         | 0/6302 [00:00<?, ?it/s]C:\Users\miket\AppData\Local\Temp\ipykernel_10072\2936267984.py:16: DeprecationWarning: `in1d` is deprecated. Use `np.isin` instead.
  keep = np.in1d(keysifv[:,-1],added)
100%|██████████████████████████████████████████████████████████████████████████████| 6302/6302 [01:55<00:00, 54.41it/s]


In [19]:
def apply_affine(pA,M):
    pA_hom = np.hstack([pA, np.ones((pA.shape[0], 1))])
    pBE = np.dot(pA_hom,M)
    return pBE
    
def get_affine(pA,pB):
    # Convert to homogeneous coordinates by adding a column of ones
    import numpy as np
    
    # Assume pA and pB are your input arrays of shape (n_points, 2)
    # For example:
    # pA = np.array([[x1, y1], [x2, y2], ...])
    # pB = np.array([[x1', y1'], [x2', y2'], ...])
    
    # Step 1: Convert points to homogeneous coordinates by adding a column of ones
    pA_hom = np.hstack([pA, np.ones((pA.shape[0], 1))])
    pB_hom = np.hstack([pB, np.ones((pB.shape[0], 1))])
    
    # Step 2: Prepare matrix A using the homogeneous coordinates of pA
    # and matrix B using the points pB (without converting to homogeneous coordinates)
    A = pA_hom
    B = pB
    
    # Step 3: Solve for the best affine transformation matrix X using least squares
    # The resulting matrix X will contain the top two rows of the affine transformation matrix.
    # The last row of the affine transformation matrix is always [0, 0, 1] for 2D transformations.
    X, residuals, rank, s = np.linalg.lstsq(A, B, rcond=None)
    
    # Append the last row [0, 0, 1] to the transformation matrix X to make it 3x3
    #affine_transformation_matrix = np.vstack([X, np.array([[0, 0, 1]])])
    
    #print("Affine transformation matrix:\n", affine_transformation_matrix)
    pBE = np.dot(pA_hom,X)
    return X,pBE

In [20]:
pos_fovs_f_save = pos_fovs_f.copy()

In [21]:
pos_fovs_f = pos_fovs_f_save.copy()
isbad = np.all((pos_fovs_f-pos_fovs)==0,axis=-1)
#M,_ = get_affine(pos_fovs[~isbad],pos_fovs_f[~isbad])
isbadI = np.where(isbad)[0]
#pos_fovs_f[isbad] = apply_affine(pos_fovs[isbad],M)
for i in isbadI:
#i = isbadI[0]
    posT = pos_fovs[i]
    dd,neigh = KDTree(pos_fovs[~isbad]).query(posT,20)
    M,_ =get_affine(pos_fovs[~isbad][neigh],pos_fovs_f[~isbad][neigh])
    pos_fovs_f[i] = apply_affine(posT[np.newaxis],M)[0]

In [22]:
fl_save = os.path.dirname(os.path.dirname(all_fls_col1[0]))+os.sep+'dic_pos_pixf.pkl'
print(fl_save)
pickle.dump(dict(zip(fovs,pos_fovs_f)),open(fl_save,'wb'))

Z:\MERFISHp\12_16_2025_BigSlideTest\dic_pos_pixf.pkl


In [ ]:
import napari
V = napari.view_points(pos_fovs,face_color='g',size=50)
V.add_points(pos_fovs_f,face_color='r',size=50)